In [24]:
import gymnasium as gym
import numpy as np
from pde_control_gym.src import NSReward
from tqdm import tqdm

# Setup Environment

In [25]:
# Set initial condition function to be zero
def getInitialCondition(X):
    u = np.zeros_like(X) 
    v = np.zeros_like(X) 
    p = np.zeros_like(X) 
    return u, v, p

# Set up boundary conditions here
boundary_condition = {
    "upper": ["Controllable", "Dirchilet"], 
    "lower": ["Dirchilet", "Dirchilet"], 
    "left": ["Dirchilet", "Dirchilet"], 
    "right": ["Dirchilet", "Dirchilet"], 
}

# Timestep and spatial step for PDE Solver
T = 0.201 # To perform 200 steps
dt = 1e-3
dx, dy = 0.05, 0.05
X, Y = 1, 1
u_target = np.load('target.npz')['u']
v_target = np.load('target.npz')['v']
desire_states = np.stack([u_target, v_target], axis=-1) # (NT, Nx, Ny, 2)
NS2DParameters = {
        "T": T, 
        "dt": dt, 
        "X": X,
        "dx": dx, 
        "Y": Y,
        "dy":dy,
        "action_dim": 1, 
        "reward_class": NSReward(0.1),
        "normalize": False, 
        "reset_init_condition_func": getInitialCondition,
        "boundary_condition": boundary_condition,
        "U_ref": desire_states, 
        "action_ref": 2.0 * np.ones(1000), 
}

# Make the NavierStokes PDE gym
env = gym.make("PDEControlGym-NavierStokes2D", **NS2DParameters)

# Test PPO

In [26]:
from stable_baselines3 import SAC, PPO
model = PPO.load("models/PPO0")
N_experiments = 50
T = 200
total_reward = 0
for i_id in tqdm(range(N_experiments)):
    obs, _ = env.reset(seed=i_id)
    for t in range(T):
        action, _states = model.predict(obs)
        obs, reward, done, _ , _  = env.step(action)
        total_reward += reward

/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
  0%|          | 0/50 [00:00<?, ?it/s]/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarnin

In [31]:
print(f"Total reward for PPO: {np.round(total_reward/N_experiments, 3)}")

Total reward for PPO: -5.37


# Test SAC

In [32]:
model = SAC.load("models/SAC0")
N_experiments = 50
T = 200
total_reward = 0
for i_id in tqdm(range(N_experiments)):
    obs, _ = env.reset(seed=i_id)
    for t in range(T):
        action, _states = model.predict(obs)
        obs, reward, done, _ , _  = env.step(action)
        total_reward += reward

/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 7.07GB > 5.82GB
  warnings.warn(
100%|██████████| 50/50 [05:14<00:00,  6.29s/it]


In [34]:
print(f"Total reward for SAC: {np.round(total_reward/N_experiments,3)}")

Total reward for SAC: -17.829


# Test Optimization

In [35]:
from pde_control_gym.src.environments2d.navier_stokes2D import central_difference, laplace
# Model-Based Optimization to optimize action 
def apply_boundary(a1, a2):
    a1[:,[-1, 0]] = 0.
    a1[[-1,0],:] = 0.
    a2[:,[-1, 0]] = 0.
    a2[[-1,0],:] = 0.
    return a1, a2

N_experiments = 50
rewards = []
for i_id in range(N_experiments):
    np.random.seed(i_id)
    total_reward = 0.
    U, V = [], []
    env.reset(seed=0)
    for t in range(T):
        obs, reward, done, _ , _ = env.step(np.random.uniform(2,4)) 
        U.append(env.u)
        V.append(env.v)
        total_reward += reward
    u_target = np.load('target.npz')['u'][1:,:,:]
    v_target = np.load('target.npz')['v'][1:,:,:]
    u_ref = [2 for _ in range(T)]
    for ite in range(1):
        Lam1, Lam2 = [], []
        Lam1.append(np.zeros_like(U[0]))
        Lam2.append(np.zeros_like(U[0]))
        pressure = np.zeros_like(U[0])
        for t in range(T-1):
            lam1, lam2 = Lam1[-1], Lam2[-1]
            dl1dx, dl1dy = central_difference(lam1,"x",dx), central_difference(lam1, "y", dy)
            dl2dx, dl2dy = central_difference(lam2,"x", dx), central_difference(lam2, "y", dy) 
            laplace_l1, laplace_l2 = laplace(lam1, dx, dy), laplace(lam2, dx, dy)
            dlam1dt = - 2 * dl1dx * U[-1-t] - dl1dy * V[-1-t] - dl2dx * V[-1-t] - 0.1 * laplace_l1 + (U[-1-t]-u_target[-1-t])
            dlam2dt = - 2 * dl2dy * V[-1-t] - dl1dy * U[-1-t] - dl2dx * U[-1-t] - 0.1 * laplace_l2 + (V[-1-t]-v_target[-1-t])
            lam1 = lam1 - dt * dlam1dt
            lam2 = lam2 - dt * dlam2dt
            lam1, lam2 = apply_boundary(lam1, lam2)
            pressure = env.solve_pressure(lam1, lam2, pressure)
            lam1 = lam1 - dt * central_difference(pressure, "x", dx)
            lam2 = lam2 - dt * central_difference(pressure, "y", dy)
            lam1, lam2 = apply_boundary(lam1, lam2)
            Lam1.append(lam1)
            Lam2.append(lam2)
        Lam1 = Lam1[::-1]
        actions = []
        for t in range(T):
            dl1dx2 = central_difference(Lam1[t], "y", dy)
            actions.append(u_ref[t] - 0.1/0.1 * sum(dl1dx2[-2, 12:17])*5*dx)
        U, V = [], []
        env.reset(seed=0)
        total_reward = 0.
        for t in tqdm(range(T)):
            obs, reward, done, _ , _ = env.step(actions[t])
            U.append(env.u)
            V.append(env.v)
            total_reward += reward
        print(total_reward)
    rewards.append(total_reward)

/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.u to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.u` for environment variables or `env.get_wrapper_attr('u')` that will search the reminding wrappers.
  logger.warn(
/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.v to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.v` for environment variables or `env.get_wrapper_attr('v')` that will search the reminding wrappers.
  logger.warn(
/Users/bb/opt/anaconda3/envs/pdegym/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.solve_pressure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.solve_pressure` for environment var

-7.930855511888277


100%|██████████| 200/200 [00:06<00:00, 31.14it/s]


-7.867054953022165


100%|██████████| 200/200 [00:06<00:00, 32.09it/s]


...

-7.97708835899874


100%|██████████| 200/200 [00:06<00:00, 31.03it/s]


-7.836973905172534


100%|██████████| 200/200 [00:06<00:00, 32.08it/s]

-8.110504573637636


In [36]:
print(f"Total reward for Optimization {np.round(np.mean(rewards), 3)}")

Total reward for Optimization -7.931
